In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Papillary_Cell_Carcinoma"
cohort = "GSE68950"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE68950"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE68950.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE68950.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE68950.csv"
json_path = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"caArray_golub-00327: Sanger cell line Affymetrix gene expression project"
!Series_summary	"The microarray gene expression pattern was studied using 798 different cancer cell lines. The cancer cell lines are obtained from different centers. Annotation information were provided in the supplementary file."
!Series_overall_design	"golub-00327"
!Series_overall_design	"Assay Type: Gene Expression"
!Series_overall_design	"Provider: Affymetrix"
!Series_overall_design	"Array Designs: HT_HG-U133A"
!Series_overall_design	"Organism: Homo sapiens (ncbitax)"
!Series_overall_design	"Tissue Sites: leukemia, Urinary tract, Lung, BiliaryTract, Autonomic Ganglion, Thyroid gland, Stomach, Breast, Pancreas, Head and Neck, Lymphoma, Colorectal, Placenta, Liver, Brain, Bone, pleura, Skin, endometrium, Ovary, cervix, Oesophagus, Connective and Soft Tissue, Muscle, Kidney, Prostate, Adrenal Gland, Eye, Testis, Smooth Muscle Tissue, Vulva, Unknow"
!Series_overall_design	"M

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Since the data has gene expression data (based on the Series title mentioning "Affymetrix gene expression project"),
# this dataset likely contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (kidney cancer), we can use disease state information (row 1)
trait_row = 1

# Looking at sample characteristics, there's no age information
age_row = None

# No gender information available
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert disease state information to binary trait values.
    1 for Kidney Papillary Cell Carcinoma, 0 for other conditions
    """
    if value is None:
        return None
    
    # Extract the value part after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the disease state values, identify kidney papillary cell carcinoma
    # Looking at disease states, we have "papillary carcinoma" and "renal cell carcinoma" 
    # and "papillary transitional cell carcinoma" that could be related to kidney papillary cell carcinoma
    if "papillary" in value.lower() and ("renal" in value.lower() or "kidney" in value.lower() or "transitional cell carcinoma" in value.lower()):
        return 1
    # If just papillary carcinoma and disease location is kidney, that's likely kidney papillary carcinoma
    elif "papillary carcinoma" in value.lower():
        return 1
    # Explicitly mark renal cell carcinoma
    elif "renal cell carcinoma" in value.lower():
        return 1
    elif "papillary transitional cell carcinoma" in value.lower():
        return 1
    # Other conditions
    else:
        return 0

def convert_age(value):
    # No age data available, function provided for consistency
    return None

def convert_gender(value):
    # No gender data available, function provided for consistency
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # Assuming clinical_data is the input dataframe
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview_data = preview_df(clinical_features_df)
    print("Preview of clinical features:")
    print(preview_data)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM1687570': [0.0], 'GSM1687571': [0.0], 'GSM1687572': [0.0], 'GSM1687573': [0.0], 'GSM1687574': [0.0], 'GSM1687575': [0.0], 'GSM1687576': [0.0], 'GSM1687577': [0.0], 'GSM1687578': [1.0], 'GSM1687579': [1.0], 'GSM1687580': [0.0], 'GSM1687581': [0.0], 'GSM1687582': [0.0], 'GSM1687583': [0.0], 'GSM1687584': [0.0], 'GSM1687585': [0.0], 'GSM1687586': [0.0], 'GSM1687587': [0.0], 'GSM1687588': [0.0], 'GSM1687589': [0.0], 'GSM1687590': [0.0], 'GSM1687591': [0.0], 'GSM1687592': [0.0], 'GSM1687593': [0.0], 'GSM1687594': [0.0], 'GSM1687595': [0.0], 'GSM1687596': [1.0], 'GSM1687597': [1.0], 'GSM1687598': [0.0], 'GSM1687599': [1.0], 'GSM1687600': [0.0], 'GSM1687601': [0.0], 'GSM1687602': [0.0], 'GSM1687603': [0.0], 'GSM1687604': [0.0], 'GSM1687605': [0.0], 'GSM1687606': [0.0], 'GSM1687607': [0.0], 'GSM1687608': [0.0], 'GSM1687609': [0.0], 'GSM1687610': [0.0], 'GSM1687611': [0.0], 'GSM1687612': [0.0], 'GSM1687613': [0.0], 'GSM1687614': [1.0], 'GSM1687615': [0.0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"caArray_golub-00327: Sanger cell line Affymetrix gene expression project"
Line 1: !Series_geo_accession	"GSE68950"
Line 2: !Series_status	"Public on May 16 2015"
Line 3: !Series_submission_date	"May 15 2015"
Line 4: !Series_last_update_date	"Jan 17 2017"
Line 5: !Series_summary	"The microarray gene expression pattern was studied using 798 different cancer cell lines. The cancer cell lines are obtained from different centers. Annotation information were provided in the supplementary file."
Line 6: !Series_overall_design	"golub-00327"
Line 7: !Series_overall_design	"Assay Type: Gene Expression"
Line 8: !Series_overall_design	"Provider: Affymetrix"
Line 9: !Series_overall_design	"Array Designs: HT_HG-U133A"
Found table marker at line 74
First few lines after marker:
"ID_REF"	"GSM1687570"	"GSM1687571"	"GSM1687572"	"GSM1687573"	"GSM1687574"	"GSM1687575"	"GSM1687576"	"GSM1687577"	"GSM1687578"	"GSM1687579"	"GSM1687580"	"GSM1687581"	"GS

Successfully extracted gene data with 22277 rows
First 20 gene IDs:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the first few rows
# These are probe IDs from an Affymetrix HT_HG-U133A array, not standard human gene symbols
# IDs like "1007_s_at", "1053_at", "117_at" are Affymetrix probe identifiers
# They need to be mapped to standard human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 17800121 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': [nan, nan, nan, nan, nan], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sap

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which keys store gene identifiers and gene symbols
# From examining the data:
# - 'ID' column in gene_annotation contains probe identifiers matching those in gene_data (e.g., "1007_s_at")
# - 'Gene Symbol' column contains human gene symbols (e.g., "DDR1", "RFC2")

# 2. Get gene mapping dataframe by extracting the probe ID and gene symbol columns
print("Creating gene mapping dataframe...")
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Created gene mapping with {len(gene_mapping)} rows")
print("\nPreview of gene mapping:")
print(preview_df(gene_mapping))

# 3. Convert probe-level measurements to gene expression data
print("\nApplying gene mapping to convert probe-level data to gene-level data...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted gene expression data contains {len(gene_data)} genes")
print("\nPreview of gene expression data (first few genes):")
print(preview_df(gene_data, n=5))

# Save the gene expression data to file
print(f"\nSaving gene expression data to {out_gene_data_file}...")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print("Gene expression data saved successfully")


Creating gene mapping dataframe...


Created gene mapping with 21248 rows

Preview of gene mapping:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}

Applying gene mapping to convert probe-level data to gene-level data...


Converted gene expression data contains 13046 genes

Preview of gene expression data (first few genes):
{'GSM1687570': [13.0, 15.4, 1.9, 2.1, 40.4], 'GSM1687571': [20.7, 3.5, 7.8, 34.9, 94.2], 'GSM1687572': [1.9, 2.1, 20.4, 0.4, 30.2], 'GSM1687573': [39.2, 37.6, 4.2, 1.3, 88.0], 'GSM1687574': [51.7, 4.0, 27.1, 16.6, 78.7], 'GSM1687575': [25.5, 24.3, 54.4, 26.0, 181.9], 'GSM1687576': [9.7, 12.2, 18.3, 22.6, 211.5], 'GSM1687577': [2.0, 20.9, 2.8, 6.4, 30.7], 'GSM1687578': [56.9, 5.1, 26.4, 10.5, 177.8], 'GSM1687579': [3.0, 6.3, 3.9, 22.8, 16.3], 'GSM1687580': [35.4, 6.6, 41.3, 4.9, 87.3], 'GSM1687581': [1.3, 14.2, 9.7, 16.8, 71.7], 'GSM1687582': [61.8, 645.8, 8.9, 18.0, 222.5], 'GSM1687583': [6.9, 15.9, 6.6, 5.6, 84.2], 'GSM1687584': [21.5, 9.4, 6.7, 11.6, 67.2], 'GSM1687585': [1.3, 175.8, 1.9, 11.4, 20.4], 'GSM1687586': [22.3, 1.4, 56.3, 12.9, 65.0], 'GSM1687587': [16.1, 0.7, 6.6, 15.5, 50.5], 'GSM1687588': [5.1, 105.2, 23.0, 7.6, 34.4], 'GSM1687589': [4.8, 27.1, 3.1, 14.1, 66.9], 'GSM1

Gene expression data saved successfully


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# This was already done and saved in the previous step, verified in the output
# We'll load it to make sure gene_data is available for the rest of the processing
print("\nLoading gene expression data...")
try:
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
except Exception as e:
    print(f"Error loading gene data: {e}")
    gene_data = pd.DataFrame()

# 2. Extract clinical features
print("\nExtracting clinical features...")
try:
    # Get the file paths for the SOFT file and matrix file again
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Obtain background information and clinical dataframe from matrix file
    background_info, clinical_data = get_background_and_clinical_data(
        matrix_file, 
        prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
        prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']
    )
    
    # For diagnostic purposes, check the unique values in each row
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    print("Sample Characteristics Dictionary:")
    print(sample_characteristics_dict)
    
    # Based on sample_characteristics_dict:
    # Row 1 contains disease state info
    # Row 2 contains disease location
    # We'll use both to identify kidney cancer samples
    trait_row = 2  # Use disease location row
    
    # Define conversion function for the trait (kidney cancer)
    def convert_trait(value):
        """
        Convert disease location to binary trait values based on kidney location.
        1 for kidney samples, 0 for other locations
        """
        if value is None:
            return None
        
        # Extract the value part after the colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Check if it's a kidney sample
        if 'kidney' in value.lower():
            return 1  # Kidney samples as cases
        else:
            return 0  # Other locations as controls

    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=None,  # No age data
        convert_age=None,
        gender_row=None,  # No gender data
        convert_gender=None
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data extracted and saved to {out_clinical_data_file}")
    
    # For diagnostic purposes
    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))
    
except Exception as e:
    print(f"Error extracting clinical features: {e}")
    selected_clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # Check if both datasets are available
    if not gene_data.empty and not selected_clinical_df.empty:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 4. Handle missing values
        print("\nHandling missing values...")
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values, data shape: {linked_data.shape}")
        
        # 5. Determine whether the trait and demographic features are biased
        print("\nChecking for bias in features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    else:
        print("Clinical data or gene data is not available. Cannot proceed with linking.")
        linked_data = pd.DataFrame()
        is_biased = True
except Exception as e:
    print(f"Error in linking data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final quality validation
print("\nPerforming final validation...")
note = ""
if 'linked_data' not in locals() or linked_data.empty:
    note = "Dataset failed processing: no usable samples remained after filtering."
    is_biased = True
    linked_data = pd.DataFrame()  # Ensure linked_data exists for validation
elif linked_data.shape[0] < 10:
    note = "Dataset has too few samples (<10) after filtering for valid analysis."
    is_biased = True
elif 'is_biased' in locals() and is_biased:
    note = "Dataset has severely biased trait distribution."

is_gene_available = 'gene_data' in locals() and not gene_data.empty
is_trait_available = 'selected_clinical_df' in locals() and not selected_clinical_df.empty

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data,
    note=note
)

# 7. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Loading gene expression data...


Loaded gene data with 13046 genes and 798 samples

Extracting clinical features...


Sample Characteristics Dictionary:
{0: ['cosmic id: 924101', 'cosmic id: 906800', 'cosmic id: 687452', 'cosmic id: 924100', 'cosmic id: 910924', 'cosmic id: 906798', 'cosmic id: 906797', 'cosmic id: 910922', 'cosmic id: 905947', 'cosmic id: 924102', 'cosmic id: 687562', 'cosmic id: 910921', 'cosmic id: 687563', 'cosmic id: 910784', 'cosmic id: 906792', 'cosmic id: 906794', 'cosmic id: 906804', 'cosmic id: 906793', 'cosmic id: 910935', 'cosmic id: 910851', 'cosmic id: 910925', 'cosmic id: 905948', 'cosmic id: 910934', 'cosmic id: 905949', 'cosmic id: 684052', 'cosmic id: 910920', 'cosmic id: 906791', 'cosmic id: 905950', 'cosmic id: 906803', 'cosmic id: 906790'], 1: ['disease state: L2 Acute Lymphoblastic Leukemia', 'disease state: NS Acute Lymphoblastic Leukemia', 'disease state: carcinoma', 'disease state: adenocarcinoma', 'disease state: transitional cell carcinoma', 'disease state: clear cell renal cell carcinoma', 'disease state: anaplastic carcinoma', 'disease state: glioblastoma 

After handling missing values, data shape: (798, 13047)

Checking for bias in features...
For the feature 'Kidney_Papillary_Cell_Carcinoma', the least common label is '1.0' with 24 occurrences. This represents 3.01% of the dataset.
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is fine.


Performing final validation...


Linked data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE68950.csv
